In [1]:
from datetime import timedelta
from rich import print
from dynaconf import Dynaconf
from arango import ArangoClient
from utinni import Context
from pathlib import Path

from tiro.plugins.utinni import TiroTSPump
from tiro.plugins.arango import ArangoAgent
from tiro import Scenario

conf = Dynaconf(settings_files=["./config/utinni_config.toml"])
scenario = Scenario.from_yaml(Path("./config/scenario.yaml"), Path("./config/use1.yaml"))

In [2]:
context = Context(**conf.on_request)
context.add_pump("tiro", TiroTSPump(scenario=scenario, **conf.influxdb))

In [3]:
table = context.tiro_table(".*Server%(CPU|Memory)Temperature", column="Server")

In [4]:
context.bind(start=-timedelta(hours=1),
             step=timedelta(minutes=10))

table["CPUTemperature"].value

,8e99500c-c194-11ec-b135-0242ac130003,8e9e2d52-c194-11ec-b135-0242ac130003,8ea29892-c194-11ec-b135-0242ac130003,8ea6e3ca-c194-11ec-b135-0242ac130003,8eab370e-c194-11ec-b135-0242ac130003,8eb22294-c194-11ec-b135-0242ac130003,8eb69338-c194-11ec-b135-0242ac130003,8ebb2eb6-c194-11ec-b135-0242ac130003,8ec012d2-c194-11ec-b135-0242ac130003,8ec4fea0-c194-11ec-b135-0242ac130003,...,90a7c216-c194-11ec-b135-0242ac130003,90abba9c-c194-11ec-b135-0242ac130003,90afaf30-c194-11ec-b135-0242ac130003,90b3aac2-c194-11ec-b135-0242ac130003,90b7aef6-c194-11ec-b135-0242ac130003,90bda0fe-c194-11ec-b135-0242ac130003,90bf9c56-c194-11ec-b135-0242ac130003,90c19704-c194-11ec-b135-0242ac130003,90c39842-c194-11ec-b135-0242ac130003,90c593fe-c194-11ec-b135-0242ac130003
2022-04-21 16:40:00+00:00,17.745714,21.585714,19.201429,22.095,24.025,20.455,20.07,19.513333,18.505,20.131667,...,20.168333,19.423333,20.718333,19.611667,20.631667,20.17,15.996667,20.245,18.971667,20.6
2022-04-21 16:50:00+00:00,17.745714,21.585714,19.201429,22.095,24.025,20.455,20.07,19.513333,18.505,20.131667,...,20.168333,19.423333,20.718333,19.611667,20.631667,20.17,15.996667,20.245,18.971667,20.6
2022-04-21 17:00:00+00:00,17.745714,21.585714,19.201429,22.095,24.025,20.455,20.07,19.513333,18.505,20.131667,...,20.168333,19.423333,20.718333,19.611667,20.631667,20.17,15.996667,20.245,18.971667,20.6
2022-04-21 17:10:00+00:00,17.745714,21.585714,19.201429,22.095,24.025,20.455,20.07,19.513333,18.505,20.131667,...,20.168333,19.423333,20.718333,19.611667,20.631667,20.17,15.996667,20.245,18.971667,20.6
2022-04-21 17:20:00+00:00,17.745714,21.585714,19.201429,22.095,24.025,20.455,20.07,19.513333,18.505,20.131667,...,20.168333,19.423333,20.718333,19.611667,20.631667,20.17,15.996667,20.245,18.971667,20.6
2022-04-21 17:30:00+00:00,17.745714,21.585714,19.201429,22.095,24.025,20.455,20.07,19.513333,18.505,20.131667,...,20.168333,19.423333,20.718333,19.611667,20.631667,20.17,15.996667,20.245,18.971667,20.6
2022-04-21 17:40:00+00:00,17.745714,21.585714,19.201429,22.095,24.025,20.455,20.07,19.513333,18.505,20.131667,...,20.168333,19.423333,20.718333,19.611667,20.631667,20.17,15.996667,20.245,18.971667,20.6


In [5]:
gdb_client = ArangoAgent(scenario, "tiro_test", "scenario",
                         ArangoClient(hosts="http://localhost:8529"),
                         auth_info=dict(password="arangodb_password"))

In [6]:
data = gdb_client.capture_status(pattern=".*Server%(CPU|Memory)Temperature")
print(data)

{
    'Room': {
        '8e994aee-c194-11ec-b135-0242ac130003': {
            'Rack': {
                '8e994f44-c194-11ec-b135-0242ac130003': {
                    'Server': {
                        '8e99500c-c194-11ec-b135-0242ac130003': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'value': 10.11,
                                    'unit': '°C',
                                    'timestamp': '2022-04-21T17:03:37.199470'
                                },
                                'CPUTemperature': {
                                    'value': 14.27,
                                    'unit': '°C',
                                    'timestamp': '2022-04-21T17:03:37.384748'
                                }
                            }
                        },
                        '8e9e2d52-c194-11ec-b135-0242ac130003': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'value': 10.82,
                                    'unit': '°C',
                                    'timestamp': '2022-04-21T17:03:37.386096'
                                },
                                'CPUTemperature': {
                                    'value': 26.75,
                                    'unit': '°C',
                                    'timestamp': '2022-04-21T17:03:37.243654'
                                }
                            }
                        },
                        '8ea29892-c194-11ec-b135-0242ac130003': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'value': 23.52,
                                    'unit': '°C',
                                    'timestamp': '2022-04-21T17:03:37.171648'
                                },
                                'CPUTemperature': {
                                    'value': 29.56,
                                    'unit': '°C',
                                    'timestamp': '2022-04-21T17:03:37.225103'
                                }
                            }
                        },
                        '8ea6e3ca-c194-11ec-b135-0242ac130003': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'value': 27.87,
                                    'unit': '°C',
                                    'timestamp': '2022-04-21T17:03:37.969766'
                                },
                                'CPUTemperature': {
                                    'value': 26.9,
                                    'unit': '°C',
                                    'timestamp': '2022-04-21T17:03:38.603283'
                                }
                            }
                        },
                        '8eab370e-c194-11ec-b135-0242ac130003': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'value': 19.55,
                                    'unit': '°C',
                                    'timestamp': '2022-04-21T17:03:38.340714'
                                },
                                'CPUTemperature': {
                                    'value': 24.99,
                                    'unit': '°C',
                                    'timestamp': '2022-04-21T17:03:38.876175'
                                }
                            }
                        }
                    }
                },
                '8eb22096-c194-11ec-b135-0242ac130003': {
                    'Server': {
                        '8eb22294-c194-11ec-b135-0242ac130003': {
                            'Telemetry': {
                                'MemoryTemperature': {
                   